# Homework 1: Locally Sensitive Hashing
Locality Sensitive Hashing (LSH) is a technique used in computer science to solve the approximate or exact Near Neighbor Search in high-dimensional spaces. It is used to find similar items in a large dataset by hashing input items so that similar items map to the same "buckets" with high probability. LSH is commonly used in recommendation systems, image and audio recognition, and data mining.

In this particular notebook we will implement a simplified version of the LSH algorithm for to compare texts and find how similar are. We will implement 4 classes which will help us to compute how similar 2 texts are. Those classes are: Shingling, CompareSets, MinHasing and CompareSignatures.

## Dataset
As a dataset, we have used the following texts:
 - Lorem Ipsum with 5 paragraphs (https://www.lipsum.com/)\[1.txt\]
 - Lorem Ipsum with 7 paragraphs (https://www.lipsum.com/)\[2.txt\]
 - Quijote de la Mancha by Miguel de Cervantes (https://www.gutenberg.org/cache/epub/60884/pg60884.txt)\[3.txt\]
 - The Adventures of Sherlock Holmes by Arthur Conan Doyle (https://www.gutenberg.org/cache/epub/1661/pg1661.txt)\[4.txt\]
 - The picture of Dorian Gray by Oscar Wilde (https://www.gutenberg.org/cache/epub/174/pg174.txt)\[5.txt\]
 - Beyond good and evil by Friedrich Wilheim Nietzsche (https://www.gutenberg.org/cache/epub/4363/pg4363.txt)\[6.txt\]

This dataset has been selected like this, so it has two text which should be fairly similar (1 & 2), a text which should be fairly different (3) and three texts which should have something in common even though they may be different (4, 5 & 6).

In this notebook we have implemented 4 classes which we will help us to compute the similarity of 2 texts. Those classes are: Shingling, CompareSets, MinHashing and CompareSignatures.

In [1]:
import os

# define the path to the dataset folder
dataset_path: str = "dataset"

# create an empty list to store the texts
texts: list[str] = []

# loop through the files in the dataset folder
for filename in os.listdir(dataset_path):
    # check if the file is a text file
    if filename.endswith(".txt"):
        # open the file and read its contents
        with open(os.path.join(dataset_path, filename), "r") as f:
            text = f.read()
            texts.append(text)

# print the texts
for i, text in enumerate(texts):
    print(f"Text {i+1}:")
    print(text)
    print("-"*132)


Text 1:
HAPTER I.


The studio was filled with the rich odour of roses, and when the light
summer wind stirred amidst the trees of the garden, there came through
the open door the heavy scent of the lilac, or the more delicate
perfume of the pink-flowering thorn.

From the corner of the divan of Persian saddle-bags on which he was
lying, smoking, as was his custom, innumerable cigarettes, Lord Henry
Wotton could just catch the gleam of the honey-sweet and honey-coloured
blossoms of a laburnum, whose tremulous branches seemed hardly able to
bear the burden of a beauty so flamelike as theirs; and now and then
the fantastic shadows of birds in flight flitted across the long
tussore-silk curtains that were stretched in front of the huge window,
producing a kind of momentary Japanese effect, and making him think of
those pallid, jade-faced painters of Tokyo who, through the medium of
an art that is necessarily immobile, seek to convey the sense of
swiftness and motion. The sullen murmur of 

## Shingling
Shingling is a technique used in text analysis to represent a document as a set of overlapping subsequences of fixed length k, called k-shingles. To compute shingles, we slide a window of size k over the document and extract the k-length substrings that fall within the window. We then store these substrings as a set, which represents the shingles of the document.

In [2]:
class Shingling():
    
    def __init__(self, k: int):
        self.k: int = k
        
    def get_shingles(self, document: str) -> set:
        """
        This method constructs k-shingles of a given length k from a given document.
        """
        shingles: set = set()
        for i in range(len(document) - self.k + 1):
            shingle: str = document[i:i+self.k]
            shingles.add(shingle)
        return shingles
    
    def get_hashed_shingles(self, document: str):
        """
        This method computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed k-shingles.
        """
        shingles: set = self.get_shingles(document)
        hashed_shingles: list[int] = [hash(shingle) for shingle in shingles]
        hashed_shingles.sort()
        return hashed_shingles

As an example to show what is shingling, we will use the following text:
> "The quick brown fox jumps over the lazy dog"

In [3]:
text = "The quick brown fox jumps over the lazy dog"
shingling: Shingling = Shingling(k=5)
shingles: list[str] = shingling.get_shingles(text)
print(shingles)
shingles_hash: list[int] = shingling.get_hashed_shingles(text)
print("-"*132)
print(shingles_hash)



{'jumps', 'k bro', ' lazy', 'umps ', 'over ', 'The q', 'ick b', 'uick ', 's ove', 'lazy ', 'the l', 'zy do', 'quick', 'er th', 'brown', 'ox ju', 'own f', ' quic', ' jump', 'azy d', 'ver t', 'rown ', ' brow', ' the ', 'fox j', 'e qui', 'n fox', 'he qu', 'y dog', ' fox ', 'ck br', 'mps o', 'wn fo', 'ps ov', ' over', 'r the', 'e laz', 'he la', 'x jum'}
------------------------------------------------------------------------------------------------------------------------------------
[-8862180700680517981, -8589667547605000447, -8327238894082843655, -8099112370933098227, -6291577502065491005, -6263457990091811840, -5999781072920994965, -5145747959104893942, -4982578922520444881, -4475919928100001461, -4414198114300732654, -3716805226450683459, -2156497199780389386, -1975366601764274446, -1893428249863081089, -1828437489247221374, -1786740265326789609, -1414403258493019149, -1201794847509096933, -1182945358197744290, -339081885264319903, -214252034093940364, 511235490810460051, 172237767363

Now we do it with our text. We use a k=9 as we are analyzing text from books instead of emails. In the following lines we are doing exactly the same as we showed in the example above.

In [4]:
shingling: Shingling = Shingling(k=9)

hashed_shingles_list: list[int] = []

for text in texts:
    hashed_shingles: int = shingling.get_hashed_shingles(text)
    hashed_shingles_list.append(hashed_shingles)
    
for i, hashed_shingles in enumerate(hashed_shingles_list):
    print(f"Hashed shingles for Text {i+1}:")
    print(hashed_shingles)
    print("-"*132)


Hashed shingles for Text 1:
[-9219595437887803506, -9202556682027704870, -9202462530657928069, -9195099506200849453, -9193681230194170402, -9187714163850157645, -9180685763787431504, -9178555592214901625, -9177137842379898707, -9176939110509318724, -9172873728172829502, -9171337813096578084, -9164023698503315983, -9160973445699318064, -9160146074659341908, -9152312152836265025, -9137152107956411327, -9135127092633596004, -9116472063716971881, -9116010439518535988, -9112701302739285136, -9107012504201265795, -9091442162955313053, -9085755927408599161, -9085667893785292835, -9078427319070831248, -9070048282646364894, -9061776849958823881, -9060914884173317260, -9054727840668227252, -9046688778258687711, -9045055433264709020, -9043472372571550922, -9038586841778077849, -9037959361978018402, -9028712834150266096, -9011187274631523951, -9009190865889085027, -9007211387903342074, -9005695165438102496, -9002460509454986366, -8992913026240685940, -8989239622210352178, -8988875190282231123, -89

## CompareSets


## MinHashing

## CompareSignatures


## LSH